In [5]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

In [270]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI

from langchain.schema.output_parser import StrOutputParser
from langchain_core.tools import tool
from typing import List, Tuple
from langchain_core.pydantic_v1 import BaseModel, Field

In [10]:
inp_simple = "Joe spent $5 and Hannah spent $8 on a toilet plunger"
inp_complex = """Sarah bought groceries for $60 and a bottle of wine for $30.
Jake paid for gas, which costs $50, and bought snacks for $25.
Emily booked $150 hotel room for her and Sarah and paid for dinner, which was $90.
Jake booked his own hotel room for $80, and also bought a hotdog for Emily for $15.
Jake and Sarah also split a $20 popcorn which they shared with Emily"""

In [11]:
class Information(BaseModel):
    """List of information to extract from the passage"""
    purchases: List[str] = Field(description="A list of all unique purchases made")
    people: List[str] = Field(description="List of all names mentioned")

In [12]:
    llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0125")

In [20]:
extract_names_prompt = ChatPromptTemplate.from_template(
"""
You will be passed a passage below.

Thoroughly review the passage and extract the requested information.

Only extract the properties mentioned in the 'Information' function.

Passage:
{input}
"""
)


extract_names_chain = extract_names_prompt | llm.with_structured_output(Information)

In [21]:
info_simple = extract_names_chain.invoke(inp_simple)
info_simple

Information(purchases=['toilet plunger'], people=['Joe', 'Hannah'])

In [22]:
info_complex = extract_names_chain.invoke(inp_complex)
info_complex

Information(purchases=['groceries', 'bottle of wine', 'gas', 'snacks', 'hotel room', 'dinner', 'hotdog', 'popcorn'], people=['Sarah', 'Jake', 'Emily'])

## Fill in tables

In [263]:
@tool
def subtract(a: float, b: float) -> float:
    """Subtracts a - b."""
    return a - b

@tool
def divide(a: float, b: float) -> float:
    """Divides a / b."""
    return a / b

@tool
def multiply(a: float, b: float) -> float:
    """Multiplies a * b."""
    return a * b

@tool
def sum_floats(floats: List[float]) -> float:
    """Sums a list of floats, with the reason for calling the tool"""
    return sum(floats)

@tool
def count_names(names: List[str]) -> int:
    """Counts the number of names in the list. Useful for counting people."""
    return len(names)

tools = [subtract, divide, multiply, sum_floats, count_names]

In [264]:
fill_contribution_table_prompt = ChatPromptTemplate.from_template("""I need to create a contribution table from the following passage that outlines various expenses paid by different individuals.
The table should include each person's name, the specific items they paid for, and the amount they contributed for each item.

Instructions:

1. Extract the contributions made by each person from the passage
2. Fill in the table that lists each person, the items they paid for, and the corresponding amounts.
3. Ensure the table includes a total row that sums up the contributions for each item.

Here's a blank table for you to start from:

```
{table}
```

Passage:

{input}

Your work so far:
{agent_scratchpad}
""")

In [265]:
from langchain.agents import AgentExecutor, create_openai_tools_agent

fill_contribution_table_agent = create_openai_tools_agent(llm, tools, fill_contribution_table_prompt)
fill_contribution_table_agent_executor = AgentExecutor(agent=fill_contribution_table_agent, tools=tools, verbose=True)

In [266]:
def prep_table(info: Information) -> str:
    header_row = "| People | " + " | ".join(info.purchases) + " |"
    separator_row = "|---" + "|---" * len(info.purchases) + "|"

    # Generate the rows for each person
    rows = []
    for person in info.people:
        rows.append(f"| {person}" + " | " * len(info.purchases) + " |")
    
    rows.append(f"| **TOTAL** " + " | " * len(info.purchases) + " |")

    # Combine all parts into the final table
    table = "\n".join([header_row, separator_row] + rows)
    return table
print(prep_table(info_simple))

| People | toilet plunger |
|---|---|
| Joe |  |
| Hannah |  |
| **TOTAL**  |  |


In [267]:
contribution_table_simple = list(fill_contribution_table_agent_executor.stream({
    "input": inp_simple,
    "table": prep_table(info_simple)
}))



> Entering new AgentExecutor chain...

Invoking: `count_names` with `{'names': ['Joe', 'Hannah']}`


2
Invoking: `subtract` with `{'a': 8, 'b': 5}`


3.0
Invoking: `subtract` with `{'a': 8, 'b': 5}`


3.0
Invoking: `subtract` with `{'a': 8, 'b': 5}`


3.0
Invoking: `subtract` with `{'a': 8, 'b': 5}`


3.0
Invoking: `subtract` with `{'a': 8, 'b': 5}`


3.0To create the contribution table, we need to calculate the individual contributions for the toilet plunger based on the expenses mentioned in the passage. 

Here are the contributions made by Joe and Hannah for the toilet plunger:
- Joe: $5
- Hannah: $8

Now, let's fill in the table with the contributions:

```
| People | toilet plunger |
|---|---|
| Joe | $5 |
| Hannah | $8 |
| **TOTAL**  | $13 |
``` 

The total contribution for the toilet plunger is $13.

> Finished chain.


In [268]:
contribution_table_complex = list(fill_contribution_table_agent_executor.stream({
    "input": inp_complex,
    "table": prep_table(info_complex)
}))



> Entering new AgentExecutor chain...

Invoking: `count_names` with `{'names': ['Sarah', 'Jake', 'Emily']}`


3
Invoking: `subtract` with `{'a': 60, 'b': 30}`


30.0
Invoking: `subtract` with `{'a': 150, 'b': 90}`


60.0
Invoking: `subtract` with `{'a': 20, 'b': 15}`


5.0
Invoking: `sum_floats` with `{'floats': [60, 30]}`


90.0
Invoking: `sum_floats` with `{'floats': [50, 25]}`


75.0
Invoking: `sum_floats` with `{'floats': [150, 90]}`


240.0
Invoking: `sum_floats` with `{'floats': [80, 15]}`


95.0
Invoking: `sum_floats` with `{'floats': [20, 20, 20]}`


60.0Based on the contributions made by each person in the passage, here is the completed contribution table:

```
| People | groceries | bottle of wine | gas | snacks | hotel room | dinner | hotdog | popcorn |
|---|---|---|---|---|---|---|---|---|
| Sarah | $60 | $30 |  |  | $75 |  |  | $10 |
| Jake |  |  | $50 | $25 | $80 |  | $15 | $10 |
| Emily |  |  |  |  | $75 | $90 |  | $10 |
| **TOTAL**  | $60 | $30 | $50 | $25 | $230 | $9

In [280]:
temp = fill_contribution_table_agent_executor.invoke({
    "input": inp_complex,
    "table": prep_table(info_complex)
})



> Entering new AgentExecutor chain...

Invoking: `count_names` with `{'names': ['Sarah', 'Jake', 'Emily']}`


3
Invoking: `sum_floats` with `{'floats': [60, 30, 50, 25, 230, 90, 15, 20]}`
responded: To create the contribution table, I will extract the contributions made by each person from the passage and fill in the table with the corresponding amounts. Let's start by filling in the contributions for each person:

1. Sarah:
   - Groceries: $60
   - Bottle of wine: $30

2. Jake:
   - Gas: $50
   - Snacks: $25
   - Hotdog for Emily: $15
   - Split popcorn with Sarah and Emily: $10 (Jake's share)

3. Emily:
   - Hotel room with Sarah: $150
   - Dinner: $90
   - Share of popcorn with Jake and Sarah: $10 (Emily's share)

Now, let's fill in the table with the contributions:

```
| People | groceries | bottle of wine | gas | snacks | hotel room | dinner | hotdog | popcorn |
|---|---|---|---|---|---|---|---|---|
| Sarah | $60 | $30 |  |  |  |  |  | $10 |
| Jake |  |  | $50 | $25 | $80 |  | 

0.0Here is the completed contribution table based on the information provided:

```
| People | groceries | bottle of wine | gas | snacks | hotel room | dinner | hotdog | popcorn |
|---|---|---|---|---|---|---|---|---|
| Sarah | $60 | $30 |  |  |  |  |  | $10 |
| Jake |  |  | $50 | $25 | $80 |  | $15 | $10 |
| Emily |  |  |  |  | $150 | $90 |  | $10 |
| **TOTAL**  | $60 | $30 | $50 | $25 | $230 | $90 | $15 | $20 |
```

The total contributions for each item are as follows:
- Groceries: $60
- Bottle of wine: $30
- Gas: $50
- Snacks: $25
- Hotel room: $230
- Dinner: $90
- Hotdog: $15
- Popcorn: $20

If you need any further assistance or modifications, feel free to let me know!

> Finished chain.


| People | groceries | bottle of wine | gas | snacks | hotel room | dinner | hotdog | popcorn |
|---|---|---|---|---|---|---|---|---|
| Sarah | 60 | 30 |  |  | 75 |  |  | 10 |
| Jake |  |  | 50 | 25 | 80 |  | 15 | 10 |
| Emily |  |  |  |  | 75 | 90 |  | 10 |
| **TOTAL**  | 60 | 30 | 50 | 25 | 230 | 90 | 15 | 30 |

Make it table format

In [215]:
from langchain_core.pydantic_v1 import conlist

class MarkdownTable(BaseModel):
    """A Markdown Table described as a list of rows"""
    rows: List[conlist(float | str, min_items=1)] = Field("A list of rows")
        
def format_table(markdown_table):
    rows = []
    is_header = True
    for r in markdown_table.rows:
        rows.append("| " + " | ".join([str(cell) for cell in r]) + " |")
        
        if is_header:
            rows.append("|---" + "|---" * len(r) + "|")

        is_header = False
    return "\n".join(rows)

In [204]:
extract_to_list_format_prompt = ChatPromptTemplate.from_template("""Given the following information
and Markdown table, please extract into the information in the table.

Only extract the properties mentioned in the 'MarkdownTable' function.

The row can be made of a combination of `float` or `str`.
Extract amounts of money as `float`, empty cells as the value `0.0`, and text as `str`.
Extract the header as the first row.

Passage:
{input}
""")

extract_to_list_format_chain = extract_to_list_format_prompt | llm.with_structured_output(MarkdownTable)

In [216]:
extract_to_list_format_output_complex = extract_to_list_format_chain.invoke(contribution_table_complex[-1]["output"])
extract_to_list_format_output_complex

MarkdownTable(rows=[['People', 'groceries', 'bottle of wine', 'gas', 'snacks', 'hotel room', 'dinner', 'hotdog', 'popcorn'], ['Sarah', 60.0, 30.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0], ['Jake', 0.0, 0.0, 50.0, 25.0, 80.0, 0.0, 15.0, 10.0], ['Emily', 0.0, 0.0, 0.0, 0.0, 150.0, 90.0, 0.0, 10.0], ['TOTAL', 60.0, 30.0, 50.0, 25.0, 230.0, 90.0, 15.0, 30.0]])

In [218]:
print(format_table(extract_to_list_format_output_complex))

| People | groceries | bottle of wine | gas | snacks | hotel room | dinner | hotdog | popcorn |
|---|---|---|---|---|---|---|---|---|---|
| Sarah | 60.0 | 30.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 10.0 |
| Jake | 0.0 | 0.0 | 50.0 | 25.0 | 80.0 | 0.0 | 15.0 | 10.0 |
| Emily | 0.0 | 0.0 | 0.0 | 0.0 | 150.0 | 90.0 | 0.0 | 10.0 |
| TOTAL | 60.0 | 30.0 | 50.0 | 25.0 | 230.0 | 90.0 | 15.0 | 30.0 |


In [235]:
fill_consumption_table_prompt =  ChatPromptTemplate.from_template("""I have a table of contributions from people for various purchases during a trip.
Based on the provided passage, I need to calculate a per capita usage share table that shows how much each person benefits from each expense.

Markdown table showing how much each person contributed to the purchase:

```
{table}
```

Passage describing the purchases:

{input}

Instructions:

Distribute the cost of each expense equally among those who benefited from it.
Calculate the per capita usage share for each person for each expense.
Fill out the per capita usage share table based on these calculations.
The sum of the per capita usage share should equal the total cost of the purchase.

Return a Markdown table with the same format as the one above, with cell values replaced accordingly.

Your work so far:

{agent_scratchpad}
""")

In [236]:
fill_consumption_table_agent = create_openai_tools_agent(llm, tools, fill_consumption_table_prompt)
fill_consumption_table_agent_executor = AgentExecutor(agent=fill_consumption_table_agent, tools=tools, verbose=True)

In [237]:
fill_consumption_table_prompt.format(
    input = inp_complex,
    table = format_table(extract_to_list_format_output_complex),
    agent_scratchpad = []
)

'Human: I have a table of contributions from people for various purchases during a trip.\nBased on the provided passage, I need to calculate a per capita usage share table that shows how much each person benefits from each expense.\n\nMarkdown table showing how much each person contributed to the purchase:\n\n```\n| People | groceries | bottle of wine | gas | snacks | hotel room | dinner | hotdog | popcorn |\n|---|---|---|---|---|---|---|---|---|---|\n| Sarah | 60.0 | 30.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 10.0 |\n| Jake | 0.0 | 0.0 | 50.0 | 25.0 | 80.0 | 0.0 | 15.0 | 10.0 |\n| Emily | 0.0 | 0.0 | 0.0 | 0.0 | 150.0 | 90.0 | 0.0 | 10.0 |\n| TOTAL | 60.0 | 30.0 | 50.0 | 25.0 | 230.0 | 90.0 | 15.0 | 30.0 |\n```\n\nPassage describing the purchases:\n\nSarah bought groceries for $60 and a bottle of wine for $30.\nJake paid for gas, which costs $50, and bought snacks for $25.\nEmily booked $150 hotel room for her and Sarah and paid for dinner, which was $90.\nJake booked his own hotel room for

In [242]:
consumption_table_complex = list(fill_consumption_table_agent_executor.stream({
    "input": inp_complex,
    "table": format_table(extract_to_list_format_output_complex)
}))



> Entering new AgentExecutor chain...

Invoking: `subtract` with `{'a': 60, 'b': 30}`


30.0
Invoking: `subtract` with `{'a': 50, 'b': 25}`
responded: To calculate the per capita usage share for each person for each expense, we need to distribute the cost of each expense equally among those who benefited from it. Let's continue the calculations and fill out the per capita usage share table based on these calculations.

Here is the updated per capita usage share table:

```
| People | groceries | bottle of wine | gas | snacks | hotel room | dinner | hotdog | popcorn |
|---|---|---|---|---|---|---|---|---|---|
| Sarah | 30.0 | 15.0 | 0.0 | 0.0 | 75.0 | 45.0 | 7.5 | 10.0 |
| Jake | 0.0 | 0.0 | 25.0 | 12.5 | 115.0 | 45.0 | 7.5 | 10.0 |
| Emily | 0.0 | 0.0 | 0.0 | 0.0 | 115.0 | 45.0 | 7.5 | 10.0 |
| TOTAL | 60.0 | 30.0 | 50.0 | 25.0 | 305.0 | 135.0 | 22.5 | 30.0 |
```

Let's proceed with the calculations to fill out the remaining values in the table.

25.0
Invoking: `subtract` with `{'a':

In [243]:
consumption_table_complex

[{'actions': [ToolAgentAction(tool='subtract', tool_input={'a': 60, 'b': 30}, log="\nInvoking: `subtract` with `{'a': 60, 'b': 30}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_jXsvCVgkt7tjxVPOT8oaUCem', 'function': {'arguments': '{"a":60,"b":30}', 'name': 'subtract'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls'}, id='run-8f222f04-a344-4e11-93f7-74c6046274b8', tool_calls=[{'name': 'subtract', 'args': {'a': 60, 'b': 30}, 'id': 'call_jXsvCVgkt7tjxVPOT8oaUCem'}], tool_call_chunks=[{'name': 'subtract', 'args': '{"a":60,"b":30}', 'id': 'call_jXsvCVgkt7tjxVPOT8oaUCem', 'index': 0}])], tool_call_id='call_jXsvCVgkt7tjxVPOT8oaUCem')],
  'messages': [AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_jXsvCVgkt7tjxVPOT8oaUCem', 'function': {'arguments': '{"a":60,"b":30}', 'name': 'subtract'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls'}, id='run-

| People | groceries | bottle of wine | gas | snacks | hotel room | dinner | hotdog | popcorn |
|---|---|---|---|---|---|---|---|---|
| Sarah | 30.0 | 15.0 | 0.0 | 0.0 | 75.0 | 45.0 | 7.5 | 10.0 |
| Jake | 0.0 | 0.0 | 25.0 | 12.5 | 115.0 | 45.0 | 7.5 | 10.0 |
| Emily | 0.0 | 0.0 | 0.0 | 0.0 | 115.0 | 45.0 | 7.5 | 10.0 |
| TOTAL | 60.0 | 30.0 | 50.0 | 25.0 | 305.0 | 135.0 | 22.5 | 30.0 |

In [240]:
extract_to_list_format_chain.invoke()

MarkdownTable(rows=[['People', 'groceries', 'bottle of wine', 'gas', 'snacks', 'hotel room', 'dinner', 'hotdog', 'popcorn'], ['Sarah', 60.0, 30.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0], ['Jake', 0.0, 0.0, 50.0, 25.0, 80.0, 0.0, 15.0, 10.0], ['Emily', 0.0, 0.0, 0.0, 0.0, 150.0, 90.0, 0.0, 10.0], ['TOTAL', 60.0, 30.0, 50.0, 25.0, 230.0, 90.0, 15.0, 30.0]])

# Time to chain it all together!

The chain should look like:


input
-> extract names, extract items
-> prep the table
-> get contributions
-> extract the contributions table, get the usage
-> extract the usage table

In [293]:
def run_chain(passage):
    person_and_purchase_name_info: Information = extract_names_chain.invoke(passage)
    empty_md_table: str = prep_table(person_and_purchase_name_info)
    fill_contributions_message: str = fill_contribution_table_agent_executor.invoke(
        {"input": passage, "table": empty_md_table})
    contributions_table: MarkdownTable = extract_to_list_format_chain.invoke(fill_contributions_message["output"])
    fill_consumption_message: str = fill_consumption_table_agent_executor.invoke(
        {"input": passage, "table": format_table(contributions_table)})
    consumption_table: MarkdwonTable = extract_to_list_format_chain.invoke(fill_consumption_message["output"])
        
    return contributions_table, consumption_table

In [294]:
contr, cons = run_chain(inp_complex)



> Entering new AgentExecutor chain...

Invoking: `count_names` with `{'names': ['Sarah', 'Jake', 'Emily']}`


3
Invoking: `subtract` with `{'a': 60, 'b': 30}`


30.0
Invoking: `subtract` with `{'a': 60, 'b': 30}`


30.0
Invoking: `subtract` with `{'a': 60, 'b': 30}`


30.0
Invoking: `subtract` with `{'a': 60, 'b': 30}`


30.0
Invoking: `subtract` with `{'a': 50, 'b': 25}`


25.0
Invoking: `sum_floats` with `{'floats': [150, 90]}`


240.0
Invoking: `sum_floats` with `{'floats': [80, 15, 20]}`


115.0Based on the contributions made by each person in the passage, here is the completed contribution table:

```
| People | groceries | bottle of wine | gas | snacks | hotel room | dinner | hotdog | popcorn |
|---|---|---|---|---|---|---|---|---|
| Sarah | $60 | $30 |  |  | $75 |  |  | $10 |
| Jake |  |  | $50 | $25 | $80 |  | $15 | $10 |
| Emily |  |  |  |  | $75 | $90 |  |  |
| **TOTAL**  | $60 | $30 | $50 | $25 | $230 | $90 | $15 | $20 |
```

Let me know if you need any further assistance 

In [295]:
contr

MarkdownTable(rows=[['People', 'groceries', 'bottle of wine', 'gas', 'snacks', 'hotel room', 'dinner', 'hotdog', 'popcorn'], ['Sarah', 60.0, 30.0, 0.0, 0.0, 75.0, 0.0, 0.0, 10.0], ['Jake', 0.0, 0.0, 50.0, 25.0, 80.0, 0.0, 15.0, 10.0], ['Emily', 0.0, 0.0, 0.0, 0.0, 75.0, 90.0, 0.0, 0.0], ['TOTAL', 60.0, 30.0, 50.0, 25.0, 230.0, 90.0, 15.0, 20.0]])

In [296]:
cons

MarkdownTable(rows=[['People', 'groceries', 'bottle of wine', 'gas', 'snacks', 'hotel room', 'dinner', 'hotdog', 'popcorn'], ['Sarah', 20.0, 30.0, 0.0, 0.0, 37.5, 0.0, 0.0, 3.33], ['Jake', 20.0, 0.0, 50.0, 25.0, 0.0, 0.0, 20.0, 3.33], ['Emily', 20.0, 0.0, 0.0, 0.0, 37.5, 45.0, 0.0, 3.33], ['TOTAL', 60.0, 30.0, 50.0, 25.0, 75.0, 45.0, 20.0, 10.0]])

Let's throw it into our API!